In [2]:
user_queries = [
        "장기보험상품에서 암진단 후 지급기간은?",
        "장기보험상품에서 보장주기에 대해 알려줘",
        "장기보험상품에서 보장범위는 어떻게 되나요?", 
        "장기보험상품에서 보장 불가한 항목은 어떻게 되나요?",
        "장기보험상품에서 보장 가능한 병원등급은 어떻게 되나요?",
        "장기보험상품에서 보장하는 방식에서 몇 회까지 보장하는지 알려줘",
        "장기보험상품에서 보장하는 기준을 알려줘",
]

In [2]:
from openai import OpenAI

model = "qwen3:8b"
#일반보험_챗봇 id
# client = OpenAI(
#     api_key="ragflow-U5ZGEyNTdlNjkyODExZjBiODE2MDI0Mm", 
#     base_url=f"http://10.50.7.154:8080/api/v1/chats_openai/d0f38fb26c1811f0bbf00242ac150007"
# )
#장기보험_챗봇 id: 269d722c6c3a11f097260242ac150007
client = OpenAI(
    api_key="ragflow-U5ZGEyNTdlNjkyODExZjBiODE2MDI0Mm", 
    base_url=f"http://10.50.7.154:8080/api/v1/chats_openai/269d722c6c3a11f097260242ac150007"
)

def retrieve_and_generate(question, stream=False):
    completion = client.chat.completions.create(
        model=model,
        messages=[
            {"role": "user", "content": question},
        ],
        stream=stream,
    )

    if stream:
        full_response = ""
        for chunk in completion:
            # Extract the content from the chunk
            if hasattr(chunk.choices[0].delta, "content") and chunk.choices[0].delta.content:
                content = chunk.choices[0].delta.content
                full_response += content
                # Replace \n and \n\n with actual line breaks and print
                print(content.replace("\\n", "\n"), end="")
        print("\n\nFinal Processed Response:")
        print(full_response.replace("\\n", "\n")) 
    else:
        content = completion.choices[0].message.content
        print(content)

In [4]:
retrieve_and_generate(user_queries[0], stream=False)

<think>
Okay, the user is asking about the payment period for cancer diagnosis in a long-term insurance product. Let me check the knowledge base provided.

Looking at the documents, there are several mentions of cancer diagnosis and payment terms. The first document has entries about a 90-day waiting period for cancer claims, excluding certain types like thyroid cancer. If diagnosed within that period, the insurance doesn't pay. After 90 days, there's a possible reduction in benefits if it's within a year. 

Another entry mentions that after the waiting period, if diagnosed within a year, the benefit might be reduced. There's also information about regular check-up fees paid every six months for 10 years. The payment starts after 90 days from the contract date. 

The second document talks about different payment periods based on when the diagnosis occurs relative to the contract date. For example, if diagnosed within the first year, payments are 50% of the coverage amount for 20 instal

In [5]:
from langfuse import Langfuse

langfuse = Langfuse(
  secret_key="sk-lf-f7bfa6ca-90bc-43c2-9280-d4c83e09e3c9",
  public_key="pk-lf-a94e5377-c10e-4574-bece-813a41cf49e3",
  host="http://10.50.3.11:3000"
)

In [6]:
from langfuse.openai import OpenAI
from langchain_core.output_parsers import StrOutputParser

#장기보험_챗봇 id: 269d722c6c3a11f097260242ac150007
client = OpenAI(
	api_key="ragflow-U5ZGEyNTdlNjkyODExZjBiODE2MDI0Mm", 
    base_url=f"http://10.50.7.154:8080/api/v1/chats_openai/269d722c6c3a11f097260242ac150007"
)

def retrieve_and_generate(question, stream=False):
    completion = client.chat.completions.create(
        model="qwen3:8b",
        messages=[
            {"role": "user", "content": question},
        ],
        stream=stream,
    )
    output_parser = StrOutputParser()

    if stream:
        full_response = ""
        for chunk in completion:
            # Extract the content from the chunk
            if hasattr(chunk.choices[0].delta, "content") and chunk.choices[0].delta.content:
                content = chunk.choices[0].delta.content
                output = output_parser.parse(content)
                # 파싱된 결과 출력
                print(output, end="")
                full_response += content
        #output = output_parser.parse(full_response)
        #print(output)
    else:
        content = completion.choices[0].message.content
        output = output_parser.parse(content)
        print(output)

In [10]:
retrieve_and_generate(user_queries[1], stream=True)

<think>
Okay, the user is asking about the coverage period for long-term insurance products. Let me check the knowledge base provided.

Looking at the documents, there's a mention of renewal periods in the business methods document. For example, ID 1 and 2 talk about 20-year and 30-year renewable term products where the premium and refund rates are compared. The refund is not paid if the policy is canceled midway. Also, ID 3 mentions that when contracting for 20 or 30-year renewable terms, the company compares the premium and refund rates with other products. 

Another part from ID 7 lists procedures for disputes, including investigations by authorities and delays due to the insured's responsibility. However, this seems more about dispute resolution than coverage periods. 

The first document's ID 0 explains a method where the risk decreases over time, leading to reduced payouts if claims occur within a certain period. This might relate to how coverage periods affect payout ratios but 

In [11]:
from ragflow_sdk import RAGFlow

api_key = "ragflow-U5ZGEyNTdlNjkyODExZjBiODE2MDI0Mm"
base_url = "http://10.50.7.154:8080"

rag_object = RAGFlow(api_key=api_key, base_url=base_url)

# name으로 지정된 지식베이스 조회
dataset_ids = []
for dataset in rag_object.list_datasets(name="장기보험상품"):
    print(dataset)
    dataset_ids.append(dataset.id)

chunks = rag_object.retrieve(dataset_ids=dataset_ids, question=user_queries[0])
print(f'len: {len(chunks)}')
for c in chunks:
    #print(f'{c.document_name}-{len(c.content)}: {c.content}')
    #print(f'{c}')
    print(f'{c.document_id}-{c.id}, similarity: {c.similarity}: {c.content}')
# datasets = rag_object.list_datasets(name="장기보험상품")
# dataset_id = datasets[0].id
# print(dataset_id)
# rag_object.retrieve(dataset_ids=[dataset_id], question=user_queries[0])

{'avatar': None, 'chunk_count': 22022, 'chunk_method': 'naive', 'description': '장기보험 상품약관, 상품요약서, 사업방법서', 'document_count': 3, 'embedding_model': 'bona/bge-m3-korean:latest', 'id': 'ad61502219184da99b4e4ea453425e49', 'name': '장기보험상품', 'pagerank': 0, 'parser_config': {'auto_keywords': 0, 'auto_questions': 0, 'chunk_token_num': 512, 'delimiter': '\n!?;。；！？', 'graphrag': {'use_graphrag': False}, 'html4excel': False, 'layout_recognize': 'MinerU', 'raptor': {'use_raptor': False}}, 'permission': 'team', 'tenant_id': 'ca5b27a4499e4db7aa9300ba721d344e'}
len: 30
aa8bfc42a52b4b4aa20f8db1a245e382-38f3910e8cce43ab8c02c96efe016022, similarity: 0.4030673066207575: 약관에 의거하여 암(갑상선암, 제자리암, 기타피부암 및 경계성종양 제외)은 계약일 이후 90일의면책기간을 설정하고 있으며, 면책기간에 발생한 암진단(갑상선암, 제자리암, 기타피부암 및 경계성종양 제외)은 보험금을 지급하지 않습니다. 90일이 경과된 이후에도 일정기간(1년) 이내인 경우보험금이 삭감될 수 있습니다.
aa8bfc42a52b4b4aa20f8db1a245e382-3b1b79fb7169402f81a4b4b2e2198492, similarity: 0.40276745813652215: 1. 암진단(유사암제외)정기검진비(10년)(6개월지급형) : 이 특별약관의 보험기간 중에 제3조(보험금 지급에 관한 

In [ ]:
from ragflow_sdk import RAGFlow

api_key = "ragflow-U5ZGEyNTdlNjkyODExZjBiODE2MDI0Mm"
base_url = "http://10.50.7.154"
rag_object = RAGFlow(api_key=api_key, base_url=f"{base_url}:9380")
assistant = rag_object.list_chats(name="장기보험_챗봇")
assistant = assistant[0]
session = assistant.create_session()    

print("\n==================== 장기보험 챗봇 =====================\n")
print("Hello. What can I do for you?")

while True:
    question = input("\n==================== User =====================\n> ")
    print("\n==================== 장기보험 챗봇 =====================\n")
    
    cont = ""
    for ans in session.ask(question, stream=True):
        print(ans.content[len(cont):], end='', flush=True)
        cont = ans.content


==================== 장기보험 챗봇 =====================

Hello. What can I do for you?

==================== 장기보험 챗봇 =====================

<think>
Okay, the user is asking about the payment period for cancer diagnosis in a long-term insurance product. Let me check the knowledge base provided.

Looking at the documents, there are several mentions of cancer diagnosis and payment terms. The first document has entries about a 90-day waiting period for cancer claims, and after that, there's a possible 1-year period where payments might be reduced. Another entry mentions a 10-year payment period with monthly installments. There's also a table indicating different payment amounts based on when the diagnosis occurs relative to the contract start date. 

So, the key points are: after the 90-day waiting period, there's a 1-year period where payments might be reduced. Then, for claims diagnosed after the first year, the payment is full. The payment duration is 10 years with monthly installments. Als

In [82]:
from ragflow_sdk import RAGFlow
import re

api_key = "ragflow-U5ZGEyNTdlNjkyODExZjBiODE2MDI0Mm"
base_url = "http://10.50.7.154"
rag_object = RAGFlow(api_key=api_key, base_url=f"{base_url}:9380")
assistant = rag_object.list_chats(name="장기보험_챗봇")
assistant = assistant[0]
session = assistant.create_session()  

def filter_think_tags(text):
    """<think> 태그와 그 내용을 제거하는 함수"""
    # <think>...</think> 패턴을 제거
    filtered = re.sub(r'<think>.*?</think>', '', text, flags=re.DOTALL)
    return filtered.strip()

question = user_queries[0]
final_answer = None
displayed_chars = 0  # 이미 출력된 문자 수 추적

print("==================== 답변 스트리밍 ====================")
cont = ""
for ans in session.ask(question, stream=True):
    # <think> 태그 필터링
    #filtered_content = filter_think_tags(ans.content)
    cont = ans.content
    
    # 이미 출력된 부분 이후의 새로운 내용만 출력
    if len(cont) > displayed_chars:
        new_content = cont[displayed_chars:]
        if new_content.strip():
            print(new_content, end='', flush=True)
            displayed_chars = len(cont)

    final_answer = ans
print("\n\n==================== 최종 결과 ====================")
answer = filter_think_tags(final_answer.content)
print(f'Answer: {answer}')
print("\n\n==================== 인용 ====================")
if hasattr(final_answer, 'reference'):
    if final_answer.reference:
        for i, ref in enumerate(final_answer.reference, 1):
            print(f"ref [{i}]: {ref}")
    else:
        print("참조 정보가 없습니다.")

==================== 답변 스트리밍 ====================
<think>
Okay, the user is asking about the payment period for cancer diagnosis<think>
Okay, the user is asking about the payment period for cancer diagnosis in a long-term insurance product. Let me check the knowledge base provided.

Looking in a long-term insurance product. Let me check the knowledge base provided.

Looking at the documents, there are several entries related to cancer diagnosis and payment terms. at the documents, there are several entries related to cancer diagnosis and payment terms. The first document has fragments with IDs 0 to 6. ID 0 The first document has fragments with IDs 0 to 6. ID 0 mentions a 90-day waiting period after the contract date, and if cancer is mentions a 90-day waiting period after the contract date, and if cancer is diagnosed within that period, no payment is made. After 90 days, there diagnosed within that period, no payment is made. After 90 days, there's a possible reduction in benefits with

In [ ]:
api_key = "ragflow-U5ZGEyNTdlNjkyODExZjBiODE2MDI0Mm"
base_url = "http://10.50.7.154"
rag_object = RAGFlow(api_key=api_key, base_url=f"{base_url}:9380")
assistant = rag_object.list_chats(name="장기보험_챗봇")
assistant = assistant[0]
session = None
for sess in assistant.list_sessions():
    print(sess.name)
    if sess.name == "044be454":
        print("Found session:", sess)
        session = sess
        break

if session == None:
    session = assistant.create_session()
    print("Created new session:", session)

question = user_queries[0]
final_answer = None
displayed_chars = 0  # 이미 출력된 문자 수 추적

print("==================== 답변 스트리밍 ====================")
cont = ""
for ans in session.ask(question, stream=True):
    # <think> 태그 필터링
    #filtered_content = filter_think_tags(ans.content)
    cont = ans.content
    
    # 이미 출력된 부분 이후의 새로운 내용만 출력
    if len(cont) > displayed_chars:
        new_content = cont[displayed_chars:]
        if new_content.strip():
            print(new_content, end='', flush=True)
            displayed_chars = len(cont)

    final_answer = ans
print("\n\n==================== 최종 결과 ====================")
answer = filter_think_tags(final_answer.content)
print(f'Answer: {answer}')
print("\n\n==================== 인용 ====================")
if hasattr(final_answer, 'reference'):
    if final_answer.reference:
        for i, ref in enumerate(final_answer.reference, 1):
            print(f"ref [{i}]: {ref}")
    else:
        print("참조 정보가 없습니다.")



In [64]:
import requests
import json
import re

# RAGFlow HTTP API 설정
api_key = "ragflow-U5ZGEyNTdlNjkyODExZjBiODE2MDI0Mm"
base_url = "http://10.50.7.154:9380"
headers = {
    "Authorization": f"Bearer {api_key}",
    "Content-Type": "application/json"
}

def get_chat_id_by_name(chat_name):
    """chat name으로 chat id를 구하는 함수"""
    url = f"{base_url}/api/v1/chats"
    params = {
        "page": 1,
        "page_size": 100,
        "name": chat_name
    }
    
    response = requests.get(url, headers=headers, params=params)
    if response.status_code == 200:
        data = response.json()
        if data.get("data") and len(data["data"]) > 0:
            chat_id = data["data"][0]["id"]
            print(f"Found chat '{chat_name}' with ID: {chat_id}")
            return chat_id
        else:
            print(f"No chat found with name: {chat_name}")
            return None
    else:
        print(f"Error getting chats: {response.status_code} - {response.text}")
        return None

def get_dataset_ids_by_name(dataset_name):
    """dataset name으로 dataset id 리스트를 구하는 함수"""
    url = f"{base_url}/api/v1/datasets"
    params = {
        "page": 1,
        "page_size": 100,
        "name": dataset_name
    }
    
    response = requests.get(url, headers=headers, params=params)
    if response.status_code == 200:
        data = response.json()
        if data.get("data") and len(data["data"]) > 0:
            dataset_ids = [dataset["id"] for dataset in data["data"]]
            print(f"Found {len(dataset_ids)} datasets with name '{dataset_name}':")
            for i, dataset in enumerate(data["data"]):
                print(f"  [{i+1}] {dataset['id']} - {dataset['name']}")
            return dataset_ids
        else:
            print(f"No datasets found with name: {dataset_name}")
            return []
    else:
        print(f"Error getting datasets: {response.status_code} - {response.text}")
        return []

def create_session(chat_id):
    """새 세션을 생성하는 함수"""
    url = f"{base_url}/api/v1/chats/{chat_id}/sessions"
    data = {
        "name": f"HTTP API Session {len(user_queries[0])}"
    }
    
    response = requests.post(url, headers=headers, json=data)
    if response.status_code == 200:
        session_id = response.json()["data"]["id"]
        print(f"Created new session: {session_id}")
        return session_id
    else:
        print(f"Error creating session: {response.status_code} - {response.text}")
        return None



In [81]:
def ask_question_stream(question, dataset_ids=None):
    """세션에서 질문하고 스트림 응답을 받는 함수"""
    global session_id
    url = f"{base_url}/api/v1/sessions/ask"
    data = {
        "question": question,
        "stream": True
    }
    
    # dataset_ids가 있으면 추가
    if dataset_ids:
        data["dataset_ids"] = dataset_ids
        print(f"Using dataset IDs: {dataset_ids}")
    
    response = requests.post(url, headers=headers, json=data, stream=True)
    
    if response.status_code == 200:
        full_answer = ""
        final_reference = {}
        last_line_len = 0
        
        print("==================== HTTP API 답변 스트리밍 ====================")
        for line in response.iter_lines():
            if line:
                try:
                    # SSE 형식에서 data: 부분 추출
                    line_str = line.decode('utf-8')
                    if line_str.startswith('data:'):
                        json_data = line_str[5:].strip()  # 'data:' 제거 및 공백 제거

                        # JSON 파싱
                        response_data = json.loads(json_data)
                        
                        # data가 true인 경우 응답 완료로 처리
                        if response_data.get("data") is True:
                            print("\n>>> 응답이 완료되었습니다.")
                            break
                        
                        if (response_data.get("code") == 0 and 
                            "data" in response_data and 
                            isinstance(response_data["data"], dict) and
                            "answer" in response_data["data"]):
                            
                            current_answer = response_data["data"]["answer"]
                            current_reference = response_data["data"].get("reference", {})
                            
                            # 기존 줄을 덮어쓰기
                            # 이전 출력보다 짧을 경우를 대비해 공백으로 덮고 커서를 앞으로 이동
                            # print(' ' * last_line_len, end='\r')
                            # <think> 태그를 포함한 현재 답변을 그대로 출력
                            # print(current_answer, end='\r', flush=True)
                            # last_line_len = len(current_answer)
                            
                            # 최종 답변과 참조 정보 업데이트
                            full_answer = current_answer
                            if current_reference:
                                final_reference = current_reference
                                
                except json.JSONDecodeError as e:
                    print(f"\n[DEBUG] JSON 파싱 오류: {e}")
                    print(f"[DEBUG] 문제가 된 데이터: {json_data[:100]}...")
                    continue
                except Exception as e:
                    print(f"\n[DEBUG] 처리 오류: {e}")
                    continue
        
        print("\n\n==================== HTTP API 최종 결과 ====================")
        return {
            "answer": full_answer,
            "reference": final_reference
        }

    else:
        print(f"Error: {response.status_code} - {response.text}")
        return None

# 실행 코드
chat_name = "장기보험_챗봇"
chat_id = get_chat_id_by_name(chat_name)

if chat_id:
    # 1. Dataset name으로 dataset ids 구하기
    dataset_name = "장기보험상품"
    dataset_ids = get_dataset_ids_by_name(dataset_name)
    
    # 2. 새 세션 생성
    if 'session_id' not in globals() or not session_id:
        session_id = create_session(chat_id)
    
    if session_id:
        # 3. 질문하기 (dataset_ids 포함)
        question = user_queries[0]
        print(f"Question: {question}")
        result = ask_question_stream(question, dataset_ids)
        
        if result:
            print(f"\n==================== 요약 ====================")
            print(f"질문: {question}")
            # <think> 태그를 제거하고 최종 답변 길이 계산
            final_answer_content = re.sub(r'<think>.*?</think>', '', result['answer'], flags=re.DOTALL).strip()
            print(f"답변 길이: {len(final_answer_content)} 문자")
            print(f"참조 문서 수: {len(result['reference'].get('chunks', []))} 개")
            print("\n--- 최종 답변 ---")
            print(final_answer_content)
            print(f'refs: {result["reference"].get("chunks", [])}')

Found chat '장기보험_챗봇' with ID: 269d722c6c3a11f097260242ac150007
Found 1 datasets with name '장기보험상품':
  [1] ad61502219184da99b4e4ea453425e49 - 장기보험상품
Question: 장기보험상품에서 암진단 후 지급기간은?
Using dataset IDs: ['ad61502219184da99b4e4ea453425e49']
==================== HTTP API 답변 스트리밍 ====================
==================== HTTP API 답변 스트리밍 ====================

>>> 응답이 완료되었습니다.


==================== HTTP API 최종 결과 ====================

==================== 요약 ====================
질문: 장기보험상품에서 암진단 후 지급기간은?
답변 길이: 678 문자
참조 문서 수: 12 개

--- 최종 답변 ---
### 암 진단 후 지급기간 ##10$$ ##9$$ ##8$$ ##11$$

암 진단 후 지급기간은 **10년**으로 정해져 있습니다. 구체적인 내용은 다음과 같습니다:

1. **지급 기간**:  
   - 암 최초 진단 확정일로부터 **10년간** 지급됩니다.   ##1$$
   - 예: 2051년 7월 1일에 암 진단 확정일이 발생하면, **2061년 7월 1일**까지 지급됩니다.

2. **지급 주기**:  
   - **6개월 간격**으로 지급됩니다.  
   - 예: 진단 확정일로부터 6개월마다 1회씩 지급됩니다.

3. **특이사항**:  
   - **다른 암(예: 위암 → 소장암 등)**이 최초 진단일 이후에 발생하더라도, 지급 기간은 **최초 암 진단 확정일**로부터 10년간 유지됩니다.  
   - **면책기간**: 약관상 암 진단 후 **90일 이내**에는 보험금이 지급되지 않습니

In [120]:
api_key = "ragflow-YyYWRhZmY0ODMyMTExZjA4OTQ3MDI0Mm"
base_url = "http://10.50.3.11:80"
headers = {
    "Authorization": f"Bearer {api_key}",
    "Content-Type": "application/json"
}
def create_session(chat_id, session_name=None):
    """새 세션을 생성하는 함수"""
    url = f"{base_url}/api/v1/chats/{chat_id}/sessions"
    data = {
        "name": session_name
    }
    
    response = requests.post(url, headers=headers, json=data)
    if response.status_code == 200:
        session_id = response.json()["data"]["id"]
        print(f"Created new session: {session_id}")
        return session_id
    else:
        print(f"Error creating session: {response.status_code} - {response.text}")
        return None

In [123]:
kb_name = "흥국화재_테스트"
chat_id = "12b0d8ec78b711f0849a0242ac1e0006"
session_name = "sdk-session-3516917b"
print(create_session(chat_id, session_name))

Created new session: e07ed17c8a4411f0872f0242c0a84006
e07ed17c8a4411f0872f0242c0a84006
